https://www.londonair.org.uk/LondonAir/API/

Docs http://api.erg.kcl.ac.uk/AirQuality/Information/Documentation/pdf

Pollutants http://api.erg.kcl.ac.uk/AirQuality/Information/Species/Json

In [1]:
import requests
import json
import pprint

def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

# HASS class

In [2]:
LOCATIONS = 'Locations'
URL = 'http://api.erg.kcl.ac.uk/AirQuality/Hourly/MonitoringIndex/GroupName=London/Json'
#SCAN_INTERVAL = timedelta(minutes=15)

# Remove authorities without any Sites ['Hounslow', 'Bromley', 'Waltham Forest', 'Barnet', 'Newham']
AUTHORITIES = [
    'Barking and Dagenham',
    'Bexley',
    'Brent',
    'Camden',
    'City of London',
    'Croydon',
    'Ealing',
    'Enfield',
    'Greenwich',
    'Hackney',
    'Hammersmith and Fulham',
    'Haringey',
    'Harrow',
    'Havering',
    'Hillingdon',
    'Islington',
    'Kensington and Chelsea',
    'Kingston',
    'Lambeth',
    'Lewisham',
    'Merton',
    'Redbridge',
    'Richmond',
    'Southwark',
    'Sutton',
    'Tower Hamlets',
    'Wandsworth',
    'Westminster']

CONFIG = {LOCATIONS: ['Merton','Richmond',]}  # A list of required locations

In [3]:
def parse_api_response(response):
    """Take in the API response. API can return dict or list of data so need to check. """
    data = dict.fromkeys(AUTHORITIES)     # Holds all data
    for authority in AUTHORITIES:
        for entry in response['HourlyAirQualityIndex']['LocalAuthority']:   # Loop over entries
            if entry['@LocalAuthorityName'] == authority:
                authority_data = []
                
                if isinstance(entry['Site'], dict):
                    entry_sites_data = [entry['Site']]  
                else:
                    entry_sites_data = entry['Site']  
                
                for site in entry_sites_data:
                    site_data = {}
                    species_data = []
                    
                    site_data['updated']   = site['@BulletinDate']
                    site_data['latitude']  = site['@Latitude']
                    site_data['longitude'] = site['@Longitude']
                    site_data['site_code'] = site['@SiteCode']
                    site_data['site_name'] = site['@SiteName'].split("-")[-1].lstrip()  
                    site_data['site_type'] = site['@SiteType']
                    
                    if isinstance(site['Species'], dict):
                        species_data = [site['Species']]  
                    else:
                        species_data = site['Species'] 
                    
                    parsed_species_data = []
                    quality_list = []
                    for species in species_data:                     
                        if species['@AirQualityBand'] != 'No data': 
                            species_dict = {}
                            species_dict['description'] = species['@SpeciesDescription']
                            species_dict['code'] = species['@SpeciesCode']
                            species_dict['quality'] = species['@AirQualityBand']
                            species_dict['index'] = species['@AirQualityIndex']
                            species_dict['summary'] = species_dict['code'] + ' is ' + species_dict['quality']
                            parsed_species_data.append(species_dict)
                            quality_list.append(species_dict['quality'])
                    
                    if not parsed_species_data:      # if no valid species data
                        parsed_species_data.append('no_species_data')
                    site_data['pollutants'] = parsed_species_data
                    
                    if quality_list:
                        site_data['pollutants_status'] = max(set(quality_list), key=quality_list.count)
                        site_data['number_of_pollutants'] = len(quality_list)
                    else:
                        site_data['pollutants_status'] = 'no_species_data' 
                        site_data['number_of_pollutants'] = 0  
                        
                    authority_data.append(site_data)
                    
                data[authority] = authority_data

    return data

In [4]:
class APIData(object):
    """Get the latest data for all authorities."""

    def __init__(self):
        """Initialize the AirData object."""
        self.data = None
       
    # Update only once in scan interval.
    #@Throttle(SCAN_INTERVAL)
    def update(self):
        """Get the latest data from TFL."""
        response = requests.get(URL)
        if response.status_code != 200:
            _LOGGER.warning("Invalid response from API")
        else:
            self.data = parse_api_response(response.json())

In [5]:
class AirSensor():   # Entity
    """Single authority air sensor"""

    ICON = 'mdi:cloud-outline'

    def __init__(self, name, APIdata):
        """Initialize the sensor."""
        self._name = name
        self._APIdata = APIdata
        self._site_data = None
        self._state = None
        self._updated = None
        
    @property
    def name(self):
        """Return the name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state
    
    @property
    def site_data(self):
        """Return the dict of sites data."""
        return self._site_data

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        attrs = {}
        attrs['updated'] = self._updated
        attrs['sites'] = len(self._site_data)
        attrs['data'] = self._site_data
        return attrs

    def update(self):
        """Update the sensor."""
        self._APIdata.update()
        self._site_data = self._APIdata.data[self._name]
        self._updated = self._site_data[0]['updated']     
        sites_status = []
        for site in self._site_data:
            if site['pollutants_status'] != 'no_species_data':
                sites_status.append(site['pollutants_status'])  
        if sites_status:
            self._state = max(set(sites_status), key=sites_status.count)
        else:
            self._state = 'no_species_data'

In [6]:
def setup_platform(config):
    """Set up the Tube sensor."""
    data = APIData()
    data.update()
    sensors = []
    for name in config[LOCATIONS]:
        sensors.append(AirSensor(name, data))
    return sensors

In [7]:
#CONFIG = {LOCATIONS: AUTHORITIES}   # check with all areas - works
CONFIG = {LOCATIONS: ['Merton', 'Westminster']}   # check with all areas - works
my_sensors = setup_platform(CONFIG)

In [8]:
for sensor in my_sensors:
    sensor.update()

In [9]:
my_sensors[1].name

'Westminster'

In [10]:
my_sensors[1].state

'Low'

In [11]:
my_sensors[1]._updated

'2017-08-02 18:00:00'

In [12]:
my_sensors[1].device_state_attributes['data']

[{'latitude': '51.52254',
  'longitude': '-0.15459',
  'number_of_pollutants': 4,
  'pollutants': [{'code': 'NO2',
    'description': 'Nitrogen Dioxide',
    'index': '1',
    'quality': 'Low',
    'summary': 'NO2 is Low'},
   {'code': 'O3',
    'description': 'Ozone',
    'index': '1',
    'quality': 'Low',
    'summary': 'O3 is Low'},
   {'code': 'PM10',
    'description': 'PM10 Particulate',
    'index': '2',
    'quality': 'Low',
    'summary': 'PM10 is Low'},
   {'code': 'SO2',
    'description': 'Sulphur Dioxide',
    'index': '1',
    'quality': 'Low',
    'summary': 'SO2 is Low'}],
  'pollutants_status': 'Low',
  'site_code': 'MY1',
  'site_name': 'Marylebone Road',
  'site_type': 'Kerbside',
  'updated': '2017-08-02 18:00:00'},
 {'latitude': '51.52254',
  'longitude': '-0.15459',
  'number_of_pollutants': 1,
  'pollutants': [{'code': 'PM10',
    'description': 'PM10 Particulate',
    'index': '2',
    'quality': 'Low',
    'summary': 'PM10 is Low'}],
  'pollutants_status': 'Lo

In [91]:
my_sensors[0].site_data[0]#['Pollutants'][0]['Summary']

{'latitude': '51.4161384794862',
 'longitude': '-0.192230805042824',
 'number_of_pollutants': 1,
 'pollutants': [{'code': 'PM10',
   'description': 'PM10 Particulate',
   'index': '2',
   'quality': 'Low',
   'summary': 'PM10 is Low'}],
 'pollutants_status': 'Low',
 'site_code': 'ME2',
 'site_name': 'Merton Road',
 'site_type': 'Roadside',
 'updated': '2017-08-02 18:00:00'}